In [28]:
import tensorflow as tf

#import matplotlib.pyplot as plt
import numpy as np
import os

import time
from skimage.io import imshow
import PIL

from IPython.display import display
tf.__version__
import pydot
import graphviz

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()


In [4]:
train_images.dtype, train_images.shape

(dtype('uint8'), (60000, 28, 28))

In [5]:
def img_to_float(img):
  return (np.float32(img)[..., None]-127.5)/127.5
def img_to_uint8(img):
  return np.uint8(img*127.5+128).clip(0, 255)[...,0]

In [6]:
train_img_f32 = img_to_float(train_images)
imshow(img_to_uint8(train_img_f32[0]))

In [7]:
BUFFER_SIZE = train_img_f32.shape[0]
BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices(train_img_f32).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose
latent_dim = 100
generator = tf.keras.Sequential([
    Dense(7*7*256, use_bias=False, input_shape=(latent_dim,)),
    BatchNormalization(),
    LeakyReLU(),
    Reshape((7, 7, 256)),
    Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
    BatchNormalization(),
    LeakyReLU(),
    Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    BatchNormalization(),
    LeakyReLU(),
    Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
]
)

In [30]:
tf.keras.utils.plot_model(generator, 'my_first_model_with_shape_info.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [25]:
from tensorflow.keras.layers import Conv2D, Dropout, Flatten
discriminator =  tf.keras.Sequential([
   Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28,28, 1)),
   LeakyReLU(),
   Dropout(0.3),
   Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
   LeakyReLU(),
   Dropout(0.3),
   Flatten(),
   Dense(1)]
)

In [26]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(generated_output):
    return loss_fn(tf.ones_like(generated_output), generated_output)

In [11]:
bce = tf.keras.losses.BinaryCrossentropy()
loss = bce([0., 0., 1., 1.], [1., 1., 1., 0.])
print('Loss: ', loss.numpy())  # Loss: 11.522857

Loss:  11.522857


In [12]:
def discriminator_loss(real_output, generated_output):
    # [1,1,...,1] with real output since it is true and we want our generated examples to look like it
    real_loss = loss_fn(tf.ones_like(real_output), real_output)

    # [0,0,...,0] with generated images since they are fake
    generated_loss = loss_fn(tf.zeros_like(generated_output), generated_output)

    total_loss = real_loss + generated_loss

    return total_loss

In [13]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [14]:
EPOCHS = 50
num_examples_to_generate = 16

# We'll re-use this random vector used to seed the generator so
# it will be easier to see the improvement over time.
random_vector_for_generation = tf.random.normal([num_examples_to_generate,
                                                 latent_dim])

In [33]:
tf.random.normal([num_examples_to_generate,latent_dim])

<tf.Tensor: id=95382, shape=(100,), dtype=float32, numpy=
array([ 1.6575307 , -1.4635975 , -0.5354438 , -0.6264311 , -0.39451995,
       -0.4735537 , -0.23207007,  0.21962954,  0.672293  , -0.46024427,
       -0.93008745,  0.10455187,  0.06926529, -1.0856293 , -0.19645248,
       -0.939739  , -0.22400475,  0.15618435, -1.0767571 , -0.47302327,
       -0.32900652, -1.5620307 , -2.976549  ,  0.02526754,  0.17738378,
        0.12472948,  0.41505784, -0.65247715, -1.2324028 ,  0.34660265,
        1.1291577 , -0.35515517,  0.05800525, -0.9041143 ,  0.9453014 ,
        1.4163826 ,  0.2143913 , -2.325552  , -1.4879212 , -0.8647559 ,
        0.53465694, -1.2800211 ,  0.46103042,  0.12445242, -0.2613729 ,
       -1.0649545 , -1.030246  , -0.56645703, -0.60191894, -0.31164458,
       -0.6245885 ,  1.350399  ,  1.2571057 , -0.63624394,  0.02386798,
       -1.1413647 ,  1.1722772 , -1.8087822 ,  0.18859407, -0.9007398 ,
       -0.4734195 , -0.7435399 , -0.13776381,  1.2734365 , -0.50148636,
      

In [15]:
@tf.function
def train_step(images):
   # generating noise from a normal distribution
      noise = tf.random.normal([BATCH_SIZE, latent_dim])
      
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
      
        real_output = discriminator(images, training=True)
        generated_output = discriminator(generated_images, training=True)
         
        gen_loss = generator_loss(generated_output)
        disc_loss = discriminator_loss(real_output, generated_output)
        
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
      return gen_loss, disc_loss

In [18]:
for epoch in range(15):
  start_time = time.time()
  for images in train_dataset:
      train_step(images)
  fake = generator(random_vector_for_generation, training=False)
  fake_concat = np.transpose(img_to_uint8(fake), [1,0,2]).reshape((28,-1))
  print(epoch, time.time()-start_time)
  display(PIL.Image.fromarray(fake_concat))
  
  

0 9.996286153793335


1 10.019220352172852


2 10.006255388259888


3 10.02520489692688


4 10.018224000930786


...

10 10.224673986434937


11 10.138996839523315


12 10.111002683639526


13 10.078089714050293


14 10.186772346496582
